In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import re
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
df = pd.read_csv('Laptop_Train_v2.csv')
df = df[df['polarity'] != 'conflict']
df

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12
...,...,...,...,...,...,...
2352,2272,We also use Paralles so we can run virtual mac...,Windows 7 Home Premium,neutral,80,102
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136


In [3]:
from nltk.corpus import stopwords
all_stop_words = set(stopwords.words("english"))

important_stop_words = [word for word in all_stop_words if ("n't" in word or "no" in word)]
stop_words = [word for word in all_stop_words if word not in important_stop_words]

def remove_stopwords(text):
    text = [word for word in text if word not in stop_words]
    return text

In [4]:
lemmatizer = WordNetLemmatizer()

def lemmatization(text):

    text = [lemmatizer.lemmatize(word) for word in text]

    return text

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocessing(text):

    text = text.lower()
    # text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[^a-z_]+$', '', text)
    text = word_tokenize(text)
    text = remove_stopwords(text)
    text = lemmatization(text)
    text = " ".join(text)

    return text

In [7]:
from tqdm import tqdm

tqdm.pandas()

df['clean_Sentence'] = df['Sentence'].progress_map(preprocessing)

df

100%|██████████| 2313/2313 [00:04<00:00, 514.34it/s] 


,id,Sentence,Aspect Term,polarity,from,to,clean_Sentence
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45,charge night skip taking cord good battery life
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86,charge night skip taking cord good battery life
2,1316,The tech guy then said the service center does...,service center,negative,27,41,tech guy said service center not 1-to-1 exchan...
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121,tech guy said service center not 1-to-1 exchan...
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12,tech guy said service center not 1-to-1 exchan...
...,...,...,...,...,...,...,...
2352,2272,We also use Paralles so we can run virtual mac...,Windows 7 Home Premium,neutral,80,102,also use paralles run virtual machine window x...
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134,also use paralles run virtual machine window x...
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170,also use paralles run virtual machine window x...
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136,"toshiba handle repair seems vary , folk indica..."


In [8]:
X = df[['clean_Sentence','Aspect Term']]

In [9]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'polarity' column
y = le.fit_transform(df['polarity'])

# Retrieve the original class labels
class_labels = le.classes_

# Create a dictionary mapping class labels to encoded values
label_mapping = {class_label: encoded_value for class_label, encoded_value in zip(class_labels, range(len(class_labels)))}

# Display the label mapping
print("Class Label Mapping:")
for class_label, encoded_value in label_mapping.items():
    print(f"{class_label}: {encoded_value}")


Class Label Mapping:
negative: 0
neutral: 1
positive: 2


In [10]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train , y_test = train_test_split(X,y, test_size=0.33)
X_train

,clean_Sentence,Aspect Term
1976,included memory computer knew vista would run ...,memory
320,video chat thing iffy im sure unpdate next ver...,video chat
761,bought laptop found tab not functioning,TAB
2022,wish toshiba would allow customer select optio...,OS
559,"bluetooth work properly , must install launch ...",Launch Manager
...,...,...
290,paint wear easily due keyboard farther back usual,paint
2139,"program ( keynote , page , number ) option sav...",programs
1055,"not experience lot heat coming , however would...",design
1680,"even , hinge may slightly tightened",hinge


In [11]:
from gensim.models import Word2Vec
# Tokenize the sentences
X_train_tokenized = X_train['clean_Sentence'].apply(word_tokenize)

model = Word2Vec(X_train_tokenized, min_count=5, vector_size=100)
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))

#summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.index_to_key)
print(words)

# save model
model.save('model3.bin')
# load model
new_model = Word2Vec.load('model3.bin')
print(new_model)

Word2Vec<vocab=746, vector_size=100, alpha=0.025>
[',', '(', 'not', 'laptop', 'computer', 'use', ')', "n't", 'window', 'great', 'battery', 'program', 'screen', 'mac', '``', 'would', 'keyboard', "'s", 'like', 'also', 'time', 'life', 'price', 'problem', 'even', 'easy', 'one', 'software', 'run', 'work', 'hard', 'no', 'good', 'feature', ':', 'drive', 'well', 'get', 'power', 'quality', 'pc', '7', 'love', 'system', 'thing', 'new', 'mouse', 'make', 'warranty', 'office', 'etc', 'hour', 'come', 'memory', 'could', 'need', 'back', 'service', 'graphic', 'performance', 'much', 'look', 'fast', 'macbook', 'better', 'machine', 'pro', 'application', 'buy', 'size', 'lot', 'apple', 'key', 'got', 'right', 'month', 'first', 'speed', 'many', '-', 'internet', 'working', 'charge', 'toshiba', 'best', 'since', 'year', '$', 'support', 'using', 'bought', 'now', 'photo', 'way', 'processor', 'day', 'money', 'pad', 'vista', 'used', 'design', '2', 'word', 'find', 'product', 'went', 'still', 'issue', 'web', 'o', 'user

In [12]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        transformed_X = []
        for words in (X['clean_Sentence'] + X['Aspect Term']):
            word_vecs = [self.word2vec[w] for w in words if w in self.word2vec]
            if not word_vecs:
                word_vecs = [np.zeros(self.dim)]
            mean_vec = np.mean(word_vecs, axis=0)
            transformed_X.append(mean_vec)
            # print(f"Length of vector: {len(mean_vec)}")  # Print the length of each vector
        return np.array(transformed_X)

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.items())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        transformed_X = []
        for words in X:
            word_vecs = [self.word2vec[w] * self.word2weight[w] for w in words if w in self.word2vec]
            if not word_vecs:
                word_vecs = [np.zeros(self.dim)]
            mean_vec = np.mean(word_vecs, axis=0)
            transformed_X.append(mean_vec)
        return np.array(transformed_X)


In [13]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

scv_clf = SVC()
DT_clf = DecisionTreeClassifier(criterion='entropy', max_depth=4)
Ada_clf = AdaBoostClassifier(n_estimators=100)
KNN_clf = KNeighborsClassifier( n_neighbors=5)
RF_clf = RandomForestClassifier()

In [14]:
from sklearn.metrics import classification_report
def print_measures(y_test, y_pred):
    print(classification_report(y_test, y_pred))

In [15]:
from sklearn.pipeline import Pipeline

#Define Pipeline
word2vec_pipeline = Pipeline([('word2vec', MeanEmbeddingVectorizer(w2v)), ('DT_clf', DT_clf)])
# Convert X_train to a 2D NumPy array
# X_train = X_train.values
#train
word2vec_pipeline.fit(X_train,y_train)

#test
print('For word2vec__pipeline')
print_measures(y_test, word2vec_pipeline.predict(X_test))

For word2vec__pipeline
              precision    recall  f1-score   support

           0       0.42      0.39      0.40       289
           1       0.37      0.05      0.08       146
           2       0.44      0.64      0.52       329

    accuracy                           0.43       764
   macro avg       0.41      0.36      0.34       764
weighted avg       0.42      0.43      0.39       764



In [16]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# Define the pipeline with best hyperparameters
best_params = {'C': 100, 'degree': 3, 'gamma': 1.0, 'kernel': 'poly'}

word2vec_pipeline = Pipeline([
    ('word2vec', MeanEmbeddingVectorizer(w2v)),
    ('svc', SVC(**best_params))
])

# Train
word2vec_pipeline.fit(X_train, y_train)

# Test
predictions = word2vec_pipeline.predict(X_test)

# Print classification report
print('For word2vec__pipeline')
print(classification_report(y_test, predictions))


For word2vec__pipeline
              precision    recall  f1-score   support

           0       0.31      0.02      0.03       289
           1       0.30      0.02      0.04       146
           2       0.44      0.98      0.60       329

    accuracy                           0.43       764
   macro avg       0.35      0.34      0.22       764
weighted avg       0.36      0.43      0.28       764

